In [10]:
import json
import time
from pathlib import Path
from typing import Callable
from local_env_variables import matrices
from Bio import Align, AlignIO, Seq, SeqIO
from local_seqtools import general_utils as tools

import numpy as np
import pandas as pd
from Bio import Align, AlignIO, Seq, SeqIO

from local_seqtools import general_utils as tools
import copy

from collections import defaultdict

In [11]:
reference_id = "9606_0:004caa"
k = 14

In [23]:
seq_dict = {
    "9606_0:004caa":"FAEAWKAQFPDSEPPRMELRSVGDIEQELERCKASIRRLEQEVNQERFRMIYLQTLLAKEKKSYDRQRWGFRRAAQAPDGASEPRASASRPQPAPADGADPPPAEEPEARPDGEGSPGKARPGTARRPGAAASGERDDRGPPASVAALRSNFERIRKGHGQPGADAEKPFYVNVEFHHERGLVKVNDKEVSDRISSLGSQAMQMERKKSQHGAGSSVGDASRPPYRGRSSESSCGVDGDYEDAELNPRFLKDNLIDANGGSRPPWPPLEYQPYQSIYVGGMMEGEGKGPLLRSQSTSEQEKRLTWPRRSYSPRSFEDCGGGYTPDCSSNENLTSSEEDFSSGQSSRVSPSPTTYRMFRDKSRSPSQNSQQSFDSSSPPTPQCHKRHRHCPVVVSEATIVGVRKTGQIWPNDGEGAFHGDADGSFGTPPGYGCAADRAEEQRRHQDGLPYIDDSPSSSPHLSSKGRGSRDALVSGALESTKASE",
    "7757_0:00087f":"FAEAWRAQFPDSEPPKMELRSLRDIEGELCSCKNSIRRLEVEVNKERFRMIYLQTLLAKEKKGYDRQRWGFRKSAQGGGGVAVPGPAGDEAGPDDDGDDPVGEVAAAAAAAAAALGGEMPGGGGGADGERPFPESPSHLRRHAADPDEGFEDTIASSRHRRAKAPDKSPLPPGLVIAPGKGAKGGEPGSPAAAGSPCKSPRVPGPPEPRARRHSSQTSEEAALCGGGGGGSRGSPGADKSSSPLQQQQQLLPPSYANVEFHAERGLVRLNEREVSDHISSLGSEAMRMERRRGGCGTASREPCVSPGRDPPQLRGPSGPGVSASSSSSSGRHHHHHHHQAASHVGYEADYEDAEPPGRFFAASPKQHPKRERPPWHVAEEATGLPESTALTDGDSTYGTGSREQLYPSSGRRQPGTGVPGLGGPPGGGGGGAGFVACKEERLSWPKQQRAHSPSSLEDAGGGYTPDCSSNDNLTSSEDELSSPGGRSGRGGGGGVGGGGAGGGVGMGLAGTGASASPTAGSSSYRLFRERENRSPSQNSQHSLDSSSPPTPHGHGTPAGLGGSGGGGGGGGRRAVRGKPSGGPGAKAVTPPGGGPDSPQRSPAGLGPKRGPPTRSRYGAPDDVFDSYPGESPFGVPSPGYVYDAERGDDGRPPPEGMPYIDDSPSSSPHAAARIRGARDAAAAAAATSAAALDAGKMGE",
    "42100_0:0032b1":"FAEAWKAQFPDSEPPRMELRSVGDIEQELDRCKASIRRLEQEVNQERFRMIYLQTLLAKEKKSYDRQRWGFRRAAQPPDGAAEPRAPRPQPAPADCSDPPPADEPEARLDGEGSPGKARPAATRRPGAAAPADHDDRGPPTSVAALRSNFERIRKGPVQPAATDAEKPFYVNVEFHHERGLVKVNDKDVSDRISSLGSQAMQMERKKSQQGAGPGPGDPPRPSYRGRLSESSCGLDGDFEDAELNPRFLKDNLINANGGSRPPWPPLEYQPYQSIYVGGMMVEGEGKGSLQRSQSTSEQEKRLTWPRRSYSPRSFEDSGGGYTPDCSSNENLTSSEEDFSSGQSSRVSPSPTTYRTFRDKSRSPSQNSQQSFDSSSPPTPQCQKRHRQGPVVVSEATIVGVRKTGQIWPSDGDSLSSRLLQDGAFHGDADAPFGTPPGYGCATDRAEEQRRHQDGLPYMDDSPSSSPHLGSKGRGSRDAPESTKVSE",
    "8255_0:002868":"FVEAWRAQFPESDPPCMELNSLGDIEQELDRCKTSIRQLEKEVNKERFRMIYLQTLLAKERKSYDGQRWGFRRVPQTRDGDQTSEPDSQRSYAGDAPVEEQQPQQQQQQRDYVKPARCKQHPEEEMDGASPAKQKGGVSLVGQDSENPEFPVGTGSVAALRSNFERIRRANSHTAGDGRGLSTMGGQEKPFYVNMEYHHERGLVKVNDRDVSDKIGTLGCQAMQIERKRSLHSLPGNLSAATGDISKAVQKSTEENCSYNGSCDDPEINPHFLKDSVIQPTGGKSERHPAECQPYTSVYVGGMMADGDTRVIHIRDHSIEEEAHLTWPRHSYSPGSFDDVGGGYTPDCSSNENLTSSEEDFSSGQSSHVSPSPTTYQMYREKSRSPSQHSQQSFESSSPPTPLSQKRLKQQVMVSEASIIGVRKTGLPWRSDGDSTTSSRTSHDNSVQGDLEAPYNETPLTYGYELEHSEEHQPHHDPLSYTGSPSSSPRLRSKSRSSRDTQSSGSLESTLSVE",
    "215358_0:000d84":"FVEAWRAQFPESDPPCMDLNSMGDIEQELDKCKTSIRHLEKEVNKERFRMIYLQTLLAKERKSYDGQRWGFKRTPQMNDGDLPTIPDNQRSHTDETAVEEHPLRGPGKTARCKPHPEGEVDGASPAKQKGGVSPVRQDSEAPDFPLGTGSVAALRSNFERIRRDSSHAAGDGRGSSEKPFYVNVEYHHERGLVKVNDREVSDKISTLGCQAMQMERKRSLHSLPGNLSAAMGDINKAVRRGRSTEGNCSYNGSYDDAEANPHFLKDNVIRSTGGKSERQPAECQPYTSVYVGGMMADGDTRIIHIRDHSIEEDAHLTWPRRSYSPGSFDDVGGGYTPDCSSNENLTSSEEDFSSGQSSHVSPSPTTYQMYREKSRSPSQHSFDSSSPPTPLSQKCLKQRAMVSEASIIRGQIWPSDGDSTTSSRTSHDNSLQGDLEAPYNETPPSYGYDLEHSEEHQPRHDPLSFTGSLESTLSVE",
    "13616_0:00197d":"FERHWRAEFPGEPVPAMQLDSVPAMQEELERCKLNLKRLQQVLAEEKFKVIYLQTALAREKRGYDCGRWEGGGGEGDSCSSSGPTKEEEEGVEAEEVRAPTELEHPQPRPQPQPQPRPRSRPQPRPRTRPEPHKEPSSSDSGPGGLASVPRTKSPHYVNLEFPAWRGTPGEGGAVLSLTAAIHQQLQQPQLLLRPREEQEPELELETERALGNGRRPEGSAQSPSALVSESPGASPEEGEELSPAAPEDTEPEALVAAEEGGDSDHDFEVVDLNEKFVLGHLLLAPCRFLPREDTPEKPLRAGSPHRWLHLIGGKRHRRSRDLDQLEAEGKDGRCSPLPPEEAHFRGAREHLPPWRRKKLLGVPGRDSPGHSSPERGSDCSYNSSDHEDVSSADFSYGADDYDGEGNEEQKVPPEGSETMPYIDESPTMSPQLSARSQGSGDGISPTPPEGLGPVVEAG",
    "336983_0:0004f6":"MEPLSHRGLPRLSWIDTLYSNFSYGTDEYDGEGNEEQKGPPEGSETMPYIDESPTMSPQLSARSQGGGDGVSPTPPEGLAPGVEAG",
}

In [13]:
def get_aligner(
    matrix_file_for_aligner: str|Path = matrices.MATRIX_DIR / "grantham_similarity_normx100_aligner_compatible", 
) -> Align.PairwiseAligner:
    aligner = Align.PairwiseAligner()
    aligner.open_gap_score = -100000
    aligner.extend_gap_score = -100000
    aligner.query_end_gap_score = 0.0
    aligner.mode = "global"
    aligner.substitution_matrix = Align.substitution_matrices.read(
        matrix_file_for_aligner
    )
    # print('getting aligner')
    return aligner


def kmer_align_aligner(
    kmer: str,
    sequence: str,
    aligner: Align.PairwiseAligner, 
) -> Align.Alignment:
    alignment = aligner.align(Seq.Seq(sequence), Seq.Seq(kmer))[0]
    return alignment


def first_non_dash_index(s):
    index = 0
    while index < len(s) and s[index] == '-':
        index += 1
    return index


def score_kmer_2_seq_no_gaps_v4(
    kmer: str,
    sequence: str,
    aligner: Align.PairwiseAligner,
):
    if len(sequence) < len(kmer):
        raise ValueError("The sequence is shorter than the kmer")
    aln = kmer_align_aligner(kmer, sequence, aligner=aligner)
    # bseq, bpos = get_aas_aligned_2_query_nongap_positions(aln)
    # bpos = get_position_of_query_start_in_target(aln)
    s = first_non_dash_index(aln[1])
    bseq = aln[0][s:s+len(kmer)]
    bpos = s
    return aln.score, bseq, bpos


def make_empty_kmer_ortho_df(kmers: list[str], ortholog_ids: list[str]):
    return pd.DataFrame(
        index=kmers,
        columns=ortholog_ids,
    )

In [22]:
def run_pairwise_kmer_alignment(
    seq_dict: dict[str, str],
    reference_id: str,
    k: int,
    matrix_file_for_aligner: str|Path=matrices.ALIGNER_MATRIX_FILE_DICT["grantham_similarity"],
):
    ortholog_idrs = copy.deepcopy(seq_dict)
    ref_idr = ortholog_idrs.pop(reference_id)
    # substitution_matrix_df = matrices.load_precomputed_matrix_df(scoring_matrix_name)
    kmers = tools.gen_kmers(ref_idr, k)
    kmers = list(set(kmers))
    score_df = make_empty_kmer_ortho_df(kmers, list(ortholog_idrs.keys()))
    subseq_df = make_empty_kmer_ortho_df(kmers, list(ortholog_idrs.keys()))
    pos_df = make_empty_kmer_ortho_df(kmers, list(ortholog_idrs.keys()))
    rbm_df = make_empty_kmer_ortho_df(kmers, list(ortholog_idrs.keys()))
    aligner = get_aligner(matrix_file_for_aligner=matrix_file_for_aligner)
    for kmer in kmers:
        for ortholog_id, ortholog_idr in ortholog_idrs.items():
            if len(ortholog_idr) < k:
                continue
            try:
                best_score, best_subseq, best_pos = score_kmer_2_seq_no_gaps_v4(kmer, ortholog_idr, aligner)
            except ValueError as e:
                print(e)
                continue
            score_df.loc[kmer, ortholog_id] = best_score
            subseq_df.loc[kmer, ortholog_id] = best_subseq
            pos_df.loc[kmer, ortholog_id] = best_pos
            try:
                reci_best_score, reci_best_subseq, _= score_kmer_2_seq_no_gaps_v4(
                    best_subseq,
                    ref_idr,
                    aligner,
                )
            except ValueError as e:
                print(e)
                continue
            rbm_df.loc[kmer, ortholog_id] = (reci_best_subseq==kmer)
    return score_df, subseq_df, pos_df, rbm_df

In [24]:
score_df, subseq_df, pos_df, rbm_df = run_pairwise_kmer_alignment(seq_dict, reference_id, k)

In [25]:
%timeit run_pairwise_kmer_alignment(seq_dict, reference_id, k)

903 ms ± 19.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
def score_alignment(seq1: str, seq2: str, subs_mat_df: pd.DataFrame) -> float:
    assert len(seq1) == len(seq2)
    score = 0
    for s1, s2 in zip(seq1, seq2):
        score += float(subs_mat_df.loc[s1, s2]) # type: ignore
    return score


def score_kmer_2_seq_no_gaps_old(
    kmer: str,
    sequence: str,
    substitution_matrix_df: pd.DataFrame,
):
    '''
    best_score = np.max(scores)
    best_position = np.argmax(scores)
    best_subseq = sequence[best_position : best_position + len(kmer)]
    '''
    if len(sequence) < len(kmer):
        raise ValueError("The sequence is shorter than the kmer")
    scores = []
    for i in range(len(sequence) - len(kmer) + 1):
        subseq = sequence[i : i + len(kmer)]
        # score = aln_tools.score_alignment(kmer, subseq, substitution_matrix_df)
        score = score_alignment(kmer, subseq, substitution_matrix_df)
        scores.append(score)
    return scores


def kmer_best_matches_2_ortho(
    kmer: str,
    sequence: str,
    substitution_matrix_df: pd.DataFrame,
):
    scores = score_kmer_2_seq_no_gaps_old(kmer, sequence, substitution_matrix_df)
    best_score = np.max(scores)
    best_position = np.argmax(scores)
    best_subseq = sequence[best_position : best_position + len(kmer)]
    return best_score, best_subseq, best_position

In [39]:
np.lib.stride_tricks.sliding_window_view(np.array([1, 2, 3, 4, 5, 6, 7, 8, 9]), window_shape=3)

array([[1, 2, 3],
       [2, 3, 4],
       [3, 4, 5],
       [4, 5, 6],
       [5, 6, 7],
       [6, 7, 8],
       [7, 8, 9]])

In [27]:
ortho_seq="FAEAWRAQFPDSEPPKMELRSLRDIEGELCSCKNSIRRLEVEVNKERFRMIYLQTLLAKEKKGYDRQRWGFRKSAQGGGGVAVPGPAGDEAGPDDDGDDPVGEVAAAAAAAAAALGGEMPGGGGGADGERPFPESPSHLRRHAADPDEGFEDTIASSRHRRAKAPDKSPLPPGLVIAPGKGAKGGEPGSPAAAGSPCKSPRVPGPPEPRARRHSSQTSEEAALCGGGGGGSRGSPGADKSSSPLQQQQQLLPPSYANVEFHAERGLVRLNEREVSDHISSLGSEAMRMERRRGGCGTASREPCVSPGRDPPQLRGPSGPGVSASSSSSSGRHHHHHHHQAASHVGYEADYEDAEPPGRFFAASPKQHPKRERPPWHVAEEATGLPESTALTDGDSTYGTGSREQLYPSSGRRQPGTGVPGLGGPPGGGGGGAGFVACKEERLSWPKQQRAHSPSSLEDAGGGYTPDCSSNDNLTSSEDELSSPGGRSGRGGGGGVGGGGAGGGVGMGLAGTGASASPTAGSSSYRLFRERENRSPSQNSQHSLDSSSPPTPHGHGTPAGLGGSGGGGGGGGRRAVRGKPSGGPGAKAVTPPGGGPDSPQRSPAGLGPKRGPPTRSRYGAPDDVFDSYPGESPFGVPSPGYVYDAERGDDGRPPPEGMPYIDDSPSSSPHAAARIRGARDAAAAAAATSAAALDAGKMGE"
kmer = "SVGDIEQELERCKA"
substitution_matrix_df = matrices.load_precomputed_matrix_df('grantham_similarity_norm')

In [31]:
%timeit score_kmer_2_seq_no_gaps_old(kmer, ortho_seq, substitution_matrix_df)

34.4 ms ± 48.4 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
ortho_arr = np.array(list(ortho_seq))
kmer_arr = np.array(list(kmer))

In [40]:
ortho_arr_view = np.lib.stride_tricks.sliding_window_view(ortho_arr, len(kmer))
print(ortho_arr_view)
print(kmer_arr)

[['F' 'A' 'E' ... 'S' 'E' 'P']
 ['A' 'E' 'A' ... 'E' 'P' 'P']
 ['E' 'A' 'W' ... 'P' 'P' 'K']
 ...
 ['A' 'A' 'A' ... 'G' 'K' 'M']
 ['A' 'A' 'T' ... 'K' 'M' 'G']
 ['A' 'T' 'S' ... 'M' 'G' 'E']]
['S' 'V' 'G' 'D' 'I' 'E' 'Q' 'E' 'L' 'E' 'R' 'C' 'K' 'A']


In [ ]:
def run_pairwise_kmer_alignment(
    seq_dict: dict[str, str],
    reference_id: str,
    k: int,
    matrix_file_for_aligner: str|Path=matrices.ALIGNER_MATRIX_FILE_DICT["grantham_similarity"],
):
    ortholog_idrs = copy.deepcopy(seq_dict)
    ref_idr = ortholog_idrs.pop(reference_id)
    # substitution_matrix_df = matrices.load_precomputed_matrix_df(scoring_matrix_name)
    kmers = tools.gen_kmers(ref_idr, k)
    kmers = list(set(kmers))
    score_df = make_empty_kmer_ortho_df(kmers, list(ortholog_idrs.keys()))
    subseq_df = make_empty_kmer_ortho_df(kmers, list(ortholog_idrs.keys()))
    pos_df = make_empty_kmer_ortho_df(kmers, list(ortholog_idrs.keys()))
    rbm_df = make_empty_kmer_ortho_df(kmers, list(ortholog_idrs.keys()))
    aligner = get_aligner(matrix_file_for_aligner=matrix_file_for_aligner)
    for kmer in kmers:
        for ortholog_id, ortholog_idr in ortholog_idrs.items():
            if len(ortholog_idr) < k:
                continue
            try:
                best_score, best_subseq, best_pos = score_kmer_2_seq_no_gaps_v4(kmer, ortholog_idr, aligner)
            except ValueError as e:
                print(e)
                continue
            score_df.loc[kmer, ortholog_id] = best_score
            subseq_df.loc[kmer, ortholog_id] = best_subseq
            pos_df.loc[kmer, ortholog_id] = best_pos
            try:
                reci_best_score, reci_best_subseq, _= score_kmer_2_seq_no_gaps_v4(
                    best_subseq,
                    ref_idr,
                    aligner,
                )
            except ValueError as e:
                print(e)
                continue
            rbm_df.loc[kmer, ortholog_id] = (reci_best_subseq==kmer)
    return score_df, subseq_df, pos_df, rbm_df